In [ ]:
using StreamOps

In [ ]:
macro pipeline2(ops...)
    # start with the last operation (which must specify its 'next' argument or use default)
    pipeline_expr = ops[end]

    # escape variable if symbol
    if isa(pipeline_expr, Symbol)
        pipeline_expr = :($(esc(pipeline_expr)))
    end

    # iterate in reverse order, wrapping each operation around the succeeding one
    # by setting the 'next' named argument to the succeeding operation
    for op in reverse(ops[1:end-1])
        if !(op isa Expr) || op.head != :call
            error("@pipeline expects instance creation expressions like OpReturn(), OpPrint(), etc., but got: $op")
        end

        next_set = false
        for arg in op.args[2:end]
            if isa(arg, Expr) && arg.head == :parameters
                for kwarg in arg.args
                    if isa(kwarg, Expr) && kwarg.head == :kw && kwarg.args[1] == :next
                        throw(ErrorException("Keyword argument 'next' already set in operation: $op"))
                    end
                end

                # Append 'next=OpX' to parameters block
                push!(arg.args, Expr(:kw, :next, pipeline_expr))
                next_set = true
            end
        end

        if !next_set
            # Insert named parameters block with chained operation before lambda or other args
            insert!(op.args, 2, Expr(:parameters, Expr(:kw, :next, pipeline_expr)))
        end
        
        pipeline_expr = op
    end

    pipeline_expr
end

pipe = @pipeline2 OpPrint(; print_fn=println) OpReturn()
# pipe = @pipeline2 OpReturn()
# println(pipeline(1.0))
# @code_lowered pipe(1.0)
println("result: $(pipe(1.0))")

In [ ]:
@macroexpand @pipeline2 OpFunc(x -> abs(x)^2) OpLag{Float64}(1) OpPrint()

In [ ]:
@macroexpand @pipeline2 OpFunc(x -> abs(x)^2) OpLag{Float64}(1) OpPrint()

pipe = @pipeline2 OpFunc(x -> abs(x)^2) OpLag{Float64}(1) OpPrint()
pipe(1.0)
pipe(5.0)
pipe(10.0)

In [ ]:
last_op = OpPrint()
pipe = @pipeline2 OpFunc(x -> abs(x)^2) OpLag{Float64}(1) last_op
pipe(1.0)

In [ ]:
dump(:(OpFunc(x -> abs(x)^2; next=OpLag{Float64}(1; next=OpPrint()))))